In [ ]:
11111111111111111111111111111111111111111111111111111111111111111111111111111111 # 80 characters

In [15]:
%%file multiple_cluster_engine.py
import ipyparallel as ipp
from collections import defaultdict
import os
import psutil
import time
from datetime import datetime
from tqdm import tqdm
import copy
import itertools

import logging # logging can create duplicate entries if you don't reload logging
try:
    from importlib import reload # Python 3
except: # Python 2 reload is a builtin
    pass

class MultipleClusterEngine(object):
    def __init__(self, 
                 cluster_job_name, 
                 n_cpus_list, 
                 ram_limit_in_GB,
                 wait_time_in_seconds,
                 input_file_names,
                 output_parent_dir,
                 function_to_process,
                 function_kwargs_dict): # always put function args in a dictionary
        reload(logging)
        self.cluster_job_name = cluster_job_name
        self.n_cpus_list = n_cpus_list
        self.ram_limit_in_GB = ram_limit_in_GB
        self.wait_time_in_seconds = wait_time_in_seconds
        self.output_parent_dir = output_parent_dir
        self.input_file_names = input_file_names
        self.function_to_process = lambda kwargs: function_to_process(**kwargs)
        self.function_kwargs_dict = function_kwargs_dict
        
        assert cluster_job_name, "Needs cluster name"
        assert len(n_cpus_list) > 0, "Needs the number of CPUs per cluster"
        assert os.path.isdir(self.output_parent_dir), "Output directory doesn't exist"
        assert len(self.input_file_names) > 0, "Need input files"

        # used by engine
        self.cluster_dict = {}
        self.load_balanced_view_dict = {}
        self.async_results_dict = defaultdict(list) # collects all the async_results
        self.file_to_cluster_order_dict = defaultdict(list) # remembers which file is sent to which cluster
        self.cluster_indexes = None
        self.logger_status = None
        self.logger_failure = None
        self.start_time = None
        self.end_time = None
        self.cluster_output_dir = None
        self.cluster_RAM_use_dict = {}
        self.cluster_pid_dict = {}

    def create_cluster_output_dir(self):
        subdirs = [name for name in os.listdir(self.output_parent_dir) if 
                   os.path.isdir(os.path.join(self.output_parent_dir, name))]
        existing_results_dir = []
        for subdir in subdirs:
            try:
                existing_results_dir.append(int(subdir.strip(self.cluster_job_name)))
            except ValueError:
                pass
        dir_index = max(existing_results_dir) + 1 if existing_results_dir else 0
        self.cluster_output_dir = os.path.join(self.output_parent_dir, 
                                               self.cluster_job_name + str(dir_index))
        os.makedirs(self.cluster_output_dir)
                
    def create_logger(self, logger_name, log_file):
        l = logging.getLogger(logger_name)
        fileHandler = logging.FileHandler(log_file)
        l.addHandler(fileHandler)
        l.setLevel(logging.INFO)
    
    def activate_logger(self):
        self.create_logger('status', os.path.join(self.cluster_output_dir, "status.log"))
        self.create_logger('failure', os.path.join(self.cluster_output_dir, "failure.log"))
        self.create_logger('ram_usage', os.path.join(self.cluster_output_dir, "ram_usage.log"))
        self.logger_status = logging.getLogger('status')
        self.logger_status.propagate = False
        self.logger_failure = logging.getLogger('failure')
        self.logger_failure.propagate = False
        self.logger_ram_usage = logging.getLogger('ram_usage')
        self.logger_ram_usage.propagate = False
        
    def profile_memory_for_cluster(self, cluster_id): # RAM in GB
        return sum(psutil.Process(pid).memory_info().rss for 
                   pid in self.cluster_pid_dict[cluster_id]) / 1e9
        
    def profile_memory_for_all_clusters(self):
        self.cluster_RAM_use_dict.clear()
        for jth_cluster in sorted(self.load_balanced_view_dict):
            self.cluster_RAM_use_dict[jth_cluster] = self.profile_memory_for_cluster(jth_cluster)           
            self.logger_ram_usage.info('{}: {}th cluster uses {} GB of RAM'.format(
                datetime.now(), jth_cluster, self.cluster_RAM_use_dict[jth_cluster]))
        self.logger_ram_usage.info('{}: All clusters use {} GB of RAM'.format(
                datetime.now(), sum(self.cluster_RAM_use_dict.values())))
        
    def clear_memory_on_cluster(self, cluster_id): # not as effective as imagined
        import gc
        self.cluster_dict[cluster_id][:].apply_async(gc.collect)
        
    def start_cluster(self, n_cpus, cluster_id):
        self.logger_status.info("{}: \tAttempting to start cluster job "
            "{}'s {}th cluster with {} CPUs".format(datetime.now(), 
            self.cluster_job_name, cluster_id, n_cpus))
        os.system("ipcluster start --n={} --profile={}{} --daemonize".format(
            n_cpus, self.cluster_job_name, cluster_id)) # should deprecate to use a safer OS call

        attempt_ctr = 0 
        while attempt_ctr < 3: # Attempt to connect to client 3 times
            time.sleep(10) # hard coded
            try:
                cluster = ipp.Client(profile='{}{}'.format(self.cluster_job_name, cluster_id))
            except ipp.error.TimeoutError:
                attempt_ctr += 1
            else:
                self.cluster_pid_dict[cluster_id] = cluster[:].apply_async(os.getpid).get()
                self.logger_status.info(('{}: \t\tCPU processes ready for action'
                    ': {}').format(datetime.now(), self.cluster_pid_dict[cluster_id]))
                return cluster
            # if there is any other error other than TimeoutError, then the error will be raised
            
    def start_all_clusters(self):
        self.activate_logger()
        self.logger_status.info('{}: Starting Multiple Cluster Engine'.format(datetime.now()))
        #self.logger_status.info('Attempting to start all clusters')
        for cluster_id, n_cpus in enumerate(self.n_cpus_list):
            self.cluster_dict[cluster_id] = self.start_cluster(n_cpus, cluster_id)
            self.load_balanced_view_dict[cluster_id] = self.cluster_dict[cluster_id].load_balanced_view()            
        self.start_time = datetime.now()
        self.logger_status.info('{}: All clusters started at {}'.format(datetime.now(), self.start_time))
        self.cluster_indexes = itertools.cycle(sorted(self.load_balanced_view_dict))
        
    def kill_cluster(self, cluster_id): # use better arguments
        self.logger_status.info(("{}: \tAttempting to kill cluster job {}'s {}th "
            "cluster with CPU processes: {}").format(datetime.now(), 
            self.cluster_job_name, cluster_id, self.cluster_pid_dict[cluster_id]))
        self.load_balanced_view_dict.pop(cluster_id)
        # cluster.purge_everything() # sometimes takes forever
        self.cluster_dict[cluster_id].close()
        self.cluster_dict.pop(cluster_id)
        os.system('ipcluster stop --profile={}{}'.format(self.cluster_job_name, cluster_id))
        self.logger_status.info('{}: \t\tCluster successfully killed'.format(datetime.now()))
        self.cluster_indexes = itertools.cycle(sorted(self.load_balanced_view_dict))
        time.sleep(5) # hard-coded
        
    def kill_all_clusters(self):
        self.end_time = datetime.now()
        n_surviving_clusters = len(self.cluster_dict)
        self.logger_status.info('{}: Killing all clusters'.format(datetime.now()))
        for cluster_id in sorted(self.cluster_dict):
            self.kill_cluster(cluster_id)
        self.logger_status.info('{}: All clusters have been killed'.format(datetime.now()))
        self.logger_status.info('{}: Multiple Cluster Engine shut down at {}'.format(
            datetime.now(), self.end_time))
        self.logger_status.info(("{}: Processed {} files using {} surviving "
            "clusters in {} minutes").format(datetime.now(), len(self.input_file_names), 
            n_surviving_clusters, (self.end_time - self.start_time).seconds / 60.0))
        logging.shutdown()
        
    def early_kill_cluster(self, cluster_id):
        self.logger_failure.info(("{}: Killing cluster job {}'s {}th cluster which "
            "was processing file {} due to exceeding RAM limit").format(
            datetime.now(), self.cluster_job_name, cluster_id, 
            self.file_to_cluster_order_dict[cluster_id][-1]))
        self.logger_ram_usage.info(("{}: Killing cluster job {}'s {}th cluster due "
            "to exceeding RAM limit").format(datetime.now(), self.cluster_job_name, cluster_id))        
        self.logger_status.info(("{}: Killing cluster job {}'s {}th cluster with CPU "
            "processes: {} due to exceeding RAM limit").format(datetime.now(), 
            self.cluster_job_name, cluster_id, self.cluster_pid_dict[cluster_id]))
        self.cluster_dict[cluster_id].close()
        os.system('ipcluster stop --profile={}{}'.format(self.cluster_job_name, cluster_id))
        self.load_balanced_view_dict.pop(cluster_id)
        self.cluster_dict.pop(cluster_id)
        self.async_results_dict.pop(cluster_id)
        
    def kill_cluster_if_ram_limit_exceeded(self): # only kills at max 1 cluster per method call
        if sum(self.cluster_RAM_use_dict.values()) > self.ram_limit_in_GB:
            cluster_id = sorted(self.cluster_RAM_use_dict, 
                                 key=self.cluster_RAM_use_dict.get, reverse=True)[0]
            self.early_kill_cluster(cluster_id)
            self.cluster_indexes = itertools.cycle(sorted(self.load_balanced_view_dict))
        assert len(self.load_balanced_view_dict) != 0, 'All clusters have been killed prematurely'
        
    def create_kwargs_dict_list(self, input_file_name, cluster_id, n_cpus):
        function_kwargs_dict = copy.deepcopy(self.function_kwargs_dict)
        function_kwargs_dict.update({'input_file_name': input_file_name,
                                    'cluster_output_dir': self.cluster_output_dir,
                                    'cluster_id': cluster_id,
                                    'n_cpus': n_cpus})
        function_kwargs_dict_list = []
        for cpu_id in range(n_cpus):
            function_kwargs_dict_list.append(copy.deepcopy(function_kwargs_dict))
            function_kwargs_dict_list[cpu_id]['cpu_id'] = cpu_id
        return function_kwargs_dict_list 
    
    def check_if_function_in_cluster_failed(self, jth_cluster):
        if self.async_results_dict[jth_cluster] == []: # cluster just started, so it
            return # doesn't have any files sent to the cluster yet
        else:
            exception = self.async_results_dict[jth_cluster][-1].exception()
            if exception:
                self.logger_failure.info(("{}: {}th cluster has error {} on "
                    "file {}").format(datetime.now(), jth_cluster, exception.args[0], 
                    self.file_to_cluster_order_dict[jth_cluster][-1]))
                self.async_results_dict[jth_cluster].pop()
                #self.async_results_dict.pop(jth_cluster)
                                     
    def run_clusters(self):
        small_file_ctr = 1 # determine if you want to have queue or differently ordered queue
        big_file_ctr = 0
        
        for ith_file in tqdm(range(len(self.input_file_names))):
            while True:
                time.sleep(self.wait_time_in_seconds)
                self.profile_memory_for_all_clusters()
                self.kill_cluster_if_ram_limit_exceeded()
                jth_cluster = next(self.cluster_indexes)
                
                if (not self.async_results_dict[jth_cluster][-1:]
                    or self.async_results_dict[jth_cluster][-1].done()): # check if cluster i is available                       
                    self.clear_memory_on_cluster(jth_cluster)
                    self.check_if_function_in_cluster_failed(jth_cluster) # check if previous file failed to process
                    
                    if jth_cluster == 0: # Send large files to large cluster (ALWAYS has id == 0)
                        index = big_file_ctr
                        big_file_ctr += 1
                    else: # Send small files to small clusters (ALWAYS have id > 0)
                        index = -small_file_ctr
                        small_file_ctr += 1
                                                                                   
                    kwargs_dict_list = self.create_kwargs_dict_list(
                        self.input_file_names[index],
                        jth_cluster, 
                        len(self.cluster_dict[jth_cluster].ids))                    
                    
                    async_result = self.load_balanced_view_dict[jth_cluster].map_async(
                        self.function_to_process, kwargs_dict_list)                                              
                    self.async_results_dict[jth_cluster].append(async_result)
                    self.file_to_cluster_order_dict[jth_cluster].append(self.input_file_names[index])
                    # write status to file--it will only have start times, no end times
                    self.logger_status.info(("{}: {} is the {}th file and is sent to "
                        "{}th cluster for processing").format(datetime.now(),
                        self.input_file_names[index], ith_file, jth_cluster))
                    break # break out of inner loop to determine if other clusters are available

        while not all(self.async_results_dict[jth_cluster][-1].done()
                      for jth_cluster in self.async_results_dict): # wait for all clusters to finish
            time.sleep(self.wait_time_in_seconds)
            self.profile_memory_for_all_clusters()
            self.kill_cluster_if_ram_limit_exceeded()
                
        cluster_set = set()
        for jth_cluster in self.cluster_indexes:
            if jth_cluster in cluster_set:
                break
            cluster_set.add(jth_cluster)
            self.check_if_function_in_cluster_failed(jth_cluster) # check if last file failed to process
        # async_results_dict; save to disk for later inspection? determine whether results takes too much RAM
        
    def main(self):
        self.create_cluster_output_dir()
        self.start_all_clusters()
        self.run_clusters()
        self.kill_all_clusters()

Overwriting multiple_cluster_engine.py


In [ ]:
# sorted from largest file to smallest, cluster for largest to smallest
# if necessary, recreate engine here if cluster shut down
# figure out queue vs deque; deque is better
# # write a crap load of documentation
# write shell script for configuration and installation

In [ ]:
# unittest with a mapper/reducer? after each map/reducer step, use gc.collect()

# MCE works on files. Hence, if you don't have any datafiles, then just create some empty files
# SSD for parallel reading (not HDD); determine if you are IO constrained
# RAM usage is heavier in Python 3 than Python 2; though Python 3 memory management is better
# during function failure: benefit (error type will be saved to failure.log) and weakness 
#     (it doesn't say what line code failed at so you have to debug your function outside 
#     of the MCE instance. You have to debug as if it were just calling the function by itself on some data)

In [ ]:
https://github.com/donnemartin/data-science-ipython-notebooks/tree/master/mapreduce